In [1]:
from patho_bench.ExperimentFactory import ExperimentFactory # Make sure you have installed Patho-Bench and this imports correctly

model_name = 'mean-UNI-NEW'
train_source = 'BRACS' 
task_name = 'slidelevel_coarse'

# Initialize the experiment
experiment = ExperimentFactory.linprobe( # This is linear probing, but similar APIs are available for coxnet, protonet, retrieval, and finetune
                    model_name = model_name,
                    train_source = train_source,
                    test_source = None, # Leave as default (None) to automatically use the test split of the training source
                    task_name = task_name,
                    patch_embeddings_dirs = '/data4/embedding/temp/embedding/BRACS/UNI-2', # Can be list of paths if patch features are split across multiple directories. See NOTE below.
                    pooled_embeddings_root = './_test_pooled_features',
                    splits_root = './_test_splits', # Splits are downloaded here from HuggingFace. You can also provide your own splits using the path_to_split and path_to_task_config arguments
                    path_to_split ='/home/yuhaowang/project/FMBC/downstream/Patho-Bench/BRACS_COARSE.tsv',
                    path_to_task_config = '/home/yuhaowang/project/FMBC/downstream/Patho-Bench/BRACS_COARSE.yaml',
                    combine_slides_per_patient = False, # Only relevant for patient-level tasks with multiple slides per patient. See NOTE below.
                    cost = 1,
                    balanced = False,
                    saveto = './_test_linprobe/{task_name}/{model_name}'.format(task_name=task_name, model_name=model_name),
                )
experiment.train()
experiment.test()
result = experiment.report_results(metric = 'macro-ovr-auc')

Loaded split from /home/yuhaowang/project/FMBC/downstream/Patho-Bench/BRACS_COARSE.tsv with 546 samples and 5 folds assigned.
Saving slide_id-level features to ./_test_pooled_features/by_slide_id/mean-UNI-NEW/BRACS, using mean-UNI-NEW...


Pre-pooling features:   0%|          | 0/546 [00:00<?, ?it/s, Running on GPU 1...]

Pre-pooling features: 100%|██████████| 546/546 [00:06<00:00, 83.11it/s, Running on GPU 1...] 


Running linprobe experiment with C = 1...


  0%|          | 0/5 [00:00<?, ?it/s]


No val set found. Skipping...


Running test split on 49 samples: 100%|██████████| 5/5 [00:01<00:00,  3.96it/s] 

macro-ovr-auc: 0.844 ± 0.024


In [ ]:
from patho_bench.ExperimentFactory import ExperimentFactory # Make sure you have installed Patho-Bench and this imports correctly

model_name = 'mean-UNI-2'
train_source = 'BRACS' 
task_name = 'slide_coarse'

# Initialize the experiment
experiment = ExperimentFactory.linprobe( # This is linear probing, but similar APIs are available for coxnet, protonet, retrieval, and finetune
                    model_name = model_name,
                    train_source = train_source,
                    test_source = None, # Leave as default (None) to automatically use the test split of the training source
                    task_name = task_name,
                    patch_embeddings_dirs = '/data4/embedding/BRACS/UNI-2', # Can be list of paths if patch features are split across multiple directories. See NOTE below.
                    pooled_embeddings_root = './_test_pooled_features',
                    splits_root = './_test_splits', # Splits are downloaded here from HuggingFace. You can also provide your own splits using the path_to_split and path_to_task_config arguments
                    path_to_split ='/home/yuhaowddang/project/FMBC/downstream/Patho-Bench/dataset_tsv/BRACS_COARSE.tsv',
                    path_to_task_config = '/home/yuhaowang/project/FMBC/downstream/Patho-Bench/BRACS_COARSE.yaml',
                    combine_slides_per_patient = False, # Only relevant for patient-level tasks with multiple slides per patient. See NOTE below.
                    cost = 1,
                    balanced = False,
                    saveto = './_test_linprobe/{task_name}/{model_name}'.format(task_name=task_name, model_name=model_name),
                )
experiment.train()
experiment.test()
result = experiment.report_results(metric = 'macro-ovr-auc')

Loaded split from /home/yuhaowang/project/FMBC/downstream/Patho-Bench/dataset_tsv/BRACS_COARSE.tsv with 546 samples and 5 folds assigned.
Saving slide_id-level features to ./_test_pooled_features/by_slide_id/mean-UNI-2/BRACS, using mean-UNI-2...


Pre-pooling features: 100%|██████████| 546/546 [00:00<00:00, 1386.29it/s, Sample BRACS_1003734 already pooled, skipping...]


Running linprobe experiment with C = 1...


  0%|          | 0/5 [00:00<?, ?it/s]


No val set found. Skipping...


Running test split on 109 samples: 100%|██████████| 5/5 [00:01<00:00,  3.25it/s]

macro-ovr-auc: 0.875 ± 0.007


: 

In [6]:
import h5py    
def read_assets_from_h5( h5_path: str) -> tuple:
        '''Read the assets from the h5 file'''
        assets = {}
        attrs = {}
        with h5py.File(h5_path, 'r') as f:
            for key in f.keys():
                assets[key] = f[key][:]
                if f[key].attrs is not None:
                    attrs[key] = dict(f[key].attrs)
        return assets, attrs
data,_ = read_assets_from_h5('/data4/embedding/BRACS/CHIEF_tile/BRACS_1506.h5')

In [11]:
import os
import shutil
data_dir ='/data4/embedding'
for dataset in os.listdir(data_dir):
    for model in os.listdir(os.path.join(data_dir,dataset)):
        #print(model)
        if model == 'CHIEF_tile':
            #os.removedirs(os.path.join(data_dir,dataset,model))
            shutil.rmtree(os.path.join(data_dir,dataset,model))

In [4]:
import os
import glob
data_dir = '/data4/embedding/temp'
file_list = glob.glob(os.path.join(data_dir,'**/*.h5'), recursive=True)
print(len(file_list))

In [4]:
import pandas as pd
#read tsv file
#/home/yuhaowang/project/FMBC/downstream/Patho-Bench/dataset_tsv/TCGA-BRCA-SUR-NEW.tsv
df = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/Patho-Bench/dataset_tsv/TCGA-BRCA-SUR-NEW.csv')

In [ ]:
df.dro

,case_id,slide_id,OS_days,OS_event,OS,fold_0,fold_1,fold_2,fold_3,fold_4
0,TCGA-3C-AALI-01,TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...,4004,1,7,train,train,train,train,train
1,TCGA-3C-AALJ-01,TCGA-3C-AALJ-01Z-00-DX1.777C0957-255A-42F0-9EE...,1473,1,6,train,train,train,train,test
2,TCGA-3C-AALK-01,TCGA-3C-AALK-01Z-00-DX1.4E6EB156-BB19-410F-878...,1447,1,6,train,train,train,train,test
3,TCGA-4H-AAAK-01,TCGA-4H-AAAK-01Z-00-DX1.ABF1B042-1970-4E28-867...,347,1,4,train,train,train,train,test
4,TCGA-5L-AAT0-01,TCGA-5L-AAT0-01Z-00-DX1.5E171263-30BF-4C6B-88A...,1476,1,6,train,train,train,train,test
...,...,...,...,...,...,...,...,...,...,...
952,TCGA-WT-AB44-01,TCGA-WT-AB44-01Z-00-DX1.B6ECEA7C-DA26-4B34-88C...,882,1,6,train,train,train,train,train
953,TCGA-XX-A899-01,TCGA-XX-A899-01Z-00-DX1.08FE27B7-73B8-4CE3-ACF...,466,1,4,train,train,train,train,test
954,TCGA-XX-A89A-01,TCGA-XX-A89A-01Z-00-DX1.671E2AD6-4D1A-4579-88C...,487,1,5,train,train,train,train,train
955,TCGA-Z7-A8R5-01,TCGA-Z7-A8R5-01Z-00-DX1.3BDB407F-514C-4131-B05...,3286,1,7,train,train,train,train,train


In [3]:
from patho_bench.ExperimentFactory import ExperimentFactory # Make sure you have installed Patho-Bench and this imports correctly

model_name = 'mean-UNI'
train_source = 'TCGA-BRCA' 
task_name = 'OS'

# Initialize the experiment
experiment = ExperimentFactory.coxnet( # This is linear probing, but similar APIs are available for coxnet, protonet, retrieval, and finetune
                    model_name = model_name,
                    train_source = train_source,
                    test_source = None, # Leave as default (None) to automatically use the test split of the training source
                    task_name = task_name,
                    patch_embeddings_dirs = '/data4/embedding/TCGA-BRCA/UNI', # Can be list of paths if patch features are split across multiple directories. See NOTE below.
                    pooled_embeddings_root = './_test_pooled_features',
                    splits_root = './_test_splits', # Splits are downloaded here from HuggingFace. You can also provide your own splits using the path_to_split and path_to_task_config arguments
                    path_to_split ='/home/yuhaowang/project/FMBC/downstream/Patho-Bench/dataset_tsv/TCGA-BRCA-SUR-NEW.csv',
                    path_to_task_config = '/home/yuhaowang/project/FMBC/downstream/Patho-Bench/configs/TCGA-BRCA.yaml',
                    combine_slides_per_patient = False, # Only relevant for patient-level tasks with multiple slides per patient. See NOTE below.
                    #cost = 1,
                    #balanced = False,
                    saveto = './_test_linprobe/{task_name}/{model_name}'.format(task_name=task_name, model_name=model_name),
                )
experiment.train()
experiment.test()
result = experiment.report_results(metric = 'macro-ovr-auc')

TypeError: ExperimentFactory.coxnet() missing 2 required positional arguments: 'alpha' and 'l1_ratio'